<a href="https://colab.research.google.com/github/mongbro/TIL/blob/master/9_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### keras RNN으로 BBC 기사 분류하기

1. 패키지 수입 및 파라미터 지정

In [1]:
# 패키지 수입
import numpy as np
import csv
import nltk # natural language tool kit

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Embedding
from keras.layers import Bidirectional
from time import time
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

In [2]:
# 파라미터 지정
MY_VOCAB = 5000   # 내가 사용할 단어의 수, 제일 많이 사용된 단어
MY_EMBED = 64     # 임베딩 차원
MY_HIDDEN = 100   # LSTM 셀의 규모
MY_LEN = 200      # 기사의 길이
MY_SPLIT = 0.8    # 학습용 데이터의 비율
MY_SAMPLE = 0   # 샘플 기사
MY_EPOCH = 100  # 반복 학습 수
TRAIN_MODE = 1    # 학습 모드와 평가 모드 선택

2. 데이터 처리

In [3]:
# 제외어 (stopword) 설정
nltk.download('stopwords')
MY_STOP = set(nltk.corpus.stopwords.words('english'))

print('영어 단어 제외')
print(MY_STOP)
print('제외어 개수 :', len(MY_STOP))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
영어 단어 제외
{'them', 're', 'for', 'those', 'do', 'out', "mustn't", 'needn', 'and', 'before', 'doesn', "wouldn't", 'which', 'wasn', 'theirs', 'as', 'not', 'are', 'yourself', 'very', 'shouldn', 'aren', 'after', 'where', 'themselves', 'm', 'any', 'these', 've', 'both', 'mightn', 'herself', 'between', 'were', 'but', 'our', 'than', 'what', 'hers', 'he', 'their', 'how', 'll', 'this', "needn't", 'has', 'because', 'too', "doesn't", 'no', 'why', 'against', 'above', "mightn't", "shan't", 'did', 'to', "wasn't", "it's", "don't", 'whom', 'she', 'through', 'an', 'most', 'y', 'during', 'once', "won't", 'hadn', "hadn't", 'into', 'then', 'other', 'a', 'or', 'down', 'itself', 'its', 'hasn', 'each', 'can', 'there', "you'd", "you're", 'while', 'being', 'should', 'of', 't', 'about', 'had', 'doing', 'was', 'my', 'have', 'until', 'the', "you'll", "aren't", 'own', 'isn', 'few', 'yourselves', 'at

In [4]:
# 데이터 보관 창고
original = []
articles = []
labels = []

In [5]:
# BBC 파일 읽고 처리
with open('/content/drive/MyDrive/Colab Notebooks/data/bbc-text.csv', 'r') as file:
    # 칼럼 이름 읽기
    reader = csv.reader(file)
    next(reader)

    # 기사 하나씩 처리
    for row in reader:
        # 카테고리 저장
        labels.append(row[0])

        # 원본 기사 저장
        original.append(row[1])

        # 제외어 삭제 하기
        news = row[1]
        for word in MY_STOP:
            mask = ' ' + word + ' '
            news = news.replace(mask, ' ')

        # 제외어를 뺀 기사 저장
        articles.append(news)
        
print('처리한 기사 수 :', len(articles))

처리한 기사 수 : 2225


In [6]:
# 샘플 기사 출력
print('샘플 기사 원본 >> ')
print(original[MY_SAMPLE])
print(labels[MY_SAMPLE])
print('총 단어 수 :', len(original[MY_SAMPLE].split()))

샘플 기사 원본 >> 
tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being buil

In [7]:
# 제외어 처리 결과
print('샘플 기사 제외어 삭제본 >> ')
print(articles[MY_SAMPLE])
print('총 단어 수 :', len(articles[MY_SAMPLE].split()))

샘플 기사 제외어 삭제본 >> 
tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes. us leading trend  programmes content delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us tivo uk sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take europe lack high-definition programming. people forward wind adverts  also forget abiding network channel schedules  putting together a-la-carte entertainment. us networks cable sa

In [8]:
# Tokenizer 처리
A_token = Tokenizer(num_words = MY_VOCAB,
                    oov_token = 'oov')
# oov란? 제외되지 않은 단어 중에서 사용 빈도가 적어서 5000개 단어에 포함하지 않는 단어들
#                        MY_VOCAB가 적어질수록 oov가 늘어난다

A_token.fit_on_texts(articles)
A_tokenized = A_token.texts_to_sequences(articles)  # => 텍스트를 숫자로 변환(hash function)

# 전환의 예
print(A_token.sequences_to_texts([[1]]))      # 1은 어떤 단어인가? => 'oov'(생략된 단어)
                                              # MY_VOCAB가 적어질수록 1이 늘어난다
print(A_token.sequences_to_texts([[1140]]))   # 1140은 어떤 단어인가? => 'the'
print(A_token.texts_to_sequences(['the']))    # 'the'는 어떤 숫자인가? => 1140
print(A_token.texts_to_sequences(['oov']))    # 'the'는 어떤 숫자인가? => 1140

['oov']
['present']
[[1173]]
[[1]]


In [9]:
# Token  처리 결과 출력
sample = A_tokenized[MY_SAMPLE]
print(sample)

[88, 165, 1143, 1206, 48, 1108, 727, 1, 77, 1060, 4252, 137, 173, 4113, 1331, 1297, 1583, 41, 7, 935, 88, 1, 316, 84, 19, 14, 130, 3115, 1317, 2507, 562, 406, 1263, 65, 2949, 3033, 1743, 8, 880, 740, 10, 940, 1, 9, 641, 1566, 1039, 401, 1986, 1206, 763, 48, 488, 1485, 2102, 1642, 125, 320, 114, 2731, 803, 1, 1074, 595, 10, 4399, 3833, 880, 2566, 137, 338, 173, 4113, 1, 1, 38, 66, 3204, 25, 9, 1, 18, 1383, 135, 441, 7, 128, 1384, 74, 4583, 474, 1, 88, 1039, 79, 1, 75, 2103, 56, 1, 88, 6, 1109, 606, 77, 1060, 88, 1956, 138, 149, 407, 9, 2865, 40, 139, 1207, 77, 1060, 4400, 7, 474, 1, 3116, 6, 2680, 1, 399, 1083, 1, 1362, 602, 1385, 2066, 1, 741, 9, 488, 1485, 2102, 125, 1904, 397, 881, 2067, 1608, 37, 1807, 2567, 4983, 1, 2508, 238, 9, 2621, 75, 804, 6, 1075, 1119, 139, 783, 563, 1, 126, 25, 1383, 1808, 432, 82, 941, 109, 19, 14, 18, 3382, 1, 36, 1442, 1, 22, 36, 91, 349, 2381, 36, 451, 230, 2067, 1363, 328, 1, 313, 804, 1120, 18, 2622, 1807, 1, 284, 721, 1162, 401, 2029, 387, 399, 2029,

In [10]:
# 기사 통계 내기
# 제외어 빼고 제일 긴, 짧은 기사 구하기
longest = max([len(x) for x in A_tokenized])
shortest = min([len(x) for x in A_tokenized])

print('제일 긴 기사 :', longest)
print('제일 짧은 기사 :', shortest)

# 모든 기사에서 제외어를 빼고 사용된 모든 단어 수
print('총 단어 수 :', len(A_token.word_counts))

제일 긴 기사 : 2279
제일 짧은 기사 : 50
총 단어 수 : 29698


In [11]:
# 기사 길이 맞추기
# MY_LEN보다 긴건 자르고 짧은건 무언가(0)를 더해준다
A_tokenized = pad_sequences(A_tokenized,
                            maxlen = MY_LEN,
                            padding = 'post',     # 200단어보다 짧은 기사는 뒷부분을 0으로 패딩처리
                            truncating = 'post')  # 200단어보다 긴 기사는 뒷부분 삭제

# 기사 길이 확인
longest = max([len(x) for x in A_tokenized])
shortest = min([len(x) for x in A_tokenized])

print('제일 긴 기사 :', longest)
print('제일 짧은 기사 :', shortest)

제일 긴 기사 : 200
제일 짧은 기사 : 200


In [12]:
# 라벨 tokenization
C_token = Tokenizer()
C_token.fit_on_texts(labels)
C_tokenized = C_token.texts_to_sequences(labels)

# 전환의 예
print(C_token.word_index)
print(C_tokenized)

{'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}
[[4], [2], [1], [1], [5], [3], [3], [1], [1], [5], [5], [2], [2], [3], [1], [2], [3], [1], [2], [4], [4], [4], [1], [1], [4], [1], [5], [4], [3], [5], [3], [4], [5], [5], [2], [3], [4], [5], [3], [2], [3], [1], [2], [1], [4], [5], [3], [3], [3], [2], [1], [3], [2], [2], [1], [3], [2], [1], [1], [2], [2], [1], [2], [1], [2], [4], [2], [5], [4], [2], [3], [2], [3], [1], [2], [4], [2], [1], [1], [2], [2], [1], [3], [2], [5], [3], [3], [2], [5], [2], [1], [1], [3], [1], [3], [1], [2], [1], [2], [5], [5], [1], [2], [3], [3], [4], [1], [5], [1], [4], [2], [5], [1], [5], [1], [5], [5], [3], [1], [1], [5], [3], [2], [4], [2], [2], [4], [1], [3], [1], [4], [5], [1], [2], [2], [4], [5], [4], [1], [2], [2], [2], [4], [1], [4], [2], [1], [5], [1], [4], [1], [4], [3], [2], [4], [5], [1], [2], [3], [2], [5], [3], [3], [5], [3], [2], [5], [3], [3], [5], [3], [1], [2], [3], [3], [2], [5], [1], [2], [2], [1], [4], [1], [4], [4], 

In [13]:
# 데이터 4분할
C_tokenized = np.array(C_tokenized)   # 기존의 C_tokenized는 list형식이다
X_train, X_test, Y_train, Y_test = train_test_split(A_tokenized,
                                                    C_tokenized,
                                                    train_size = MY_SPLIT,
                                                    shuffle = False)

# 데이터 모양 확인
print('학습용 입력 데이터 모양 :', X_train.shape)
print('학습용 출력 데이터 모양 :', Y_train.shape)

print('평가용 입력 데이터 모양 :', X_test.shape)
print('평가용 출력 데이터 모양 :', Y_test.shape)

학습용 입력 데이터 모양 : (1780, 200)
학습용 출력 데이터 모양 : (1780, 1)
평가용 입력 데이터 모양 : (445, 200)
평가용 출력 데이터 모양 : (445, 1)


3. 인공 신경망 구현

In [14]:
# RNN 구현
model = Sequential()

model.add(Embedding(input_dim = MY_VOCAB,       # 1 * 5000 행렬에 5000 * 64행렬을 곱해서
                    output_dim = MY_EMBED))     # 1 * 64 행렬로 만든다.

model.add(Dropout(rate = 0.5))  # 임의의 뉴런의 출력을 일부러 0으로 만드는 작업
                                # 왜? => 과적합을 방지하기 위해서

model.add(Bidirectional(LSTM(units = MY_HIDDEN)))

model.add(Dense(units = 6,                  # 왜 5가 아니라 6일까??
                activation = 'softmax'))    # 아까 output은 1~5였는데 RNN에서 맨 처음은 0이라서
                                            # units를 5로 하면 0~4 까지만 검색을 한다.
                                            # 즉 5번이 나올 수 없다.
print('RNN 요약')
model.summary()

RNN 요약
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
dropout (Dropout)            (None, None, 64)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               132000    
_________________________________________________________________
dense (Dense)                (None, 6)                 1206      
Total params: 453,206
Trainable params: 453,206
Non-trainable params: 0
_________________________________________________________________


4. 인공 신경망 학습

In [16]:
# RNN 학습
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'])

print('학습 시작')
begin = time()

model.fit(x = X_train,
          y = Y_train,
          epochs = MY_EPOCH,
          verbose = 1)

end = time()
print('학습시간 : {:.2f}초'.format(end - begin))

학습 시작
Epoch 1/100
56/56 [==============================] - 10s 24ms/step - loss: 1.6669 - acc: 0.2539
Epoch 2/100
56/56 [==============================] - 1s 21ms/step - loss: 1.3121 - acc: 0.4571
Epoch 3/100
56/56 [==============================] - 1s 21ms/step - loss: 0.7335 - acc: 0.7835
Epoch 4/100
56/56 [==============================] - 1s 21ms/step - loss: 0.5618 - acc: 0.8125
Epoch 5/100
56/56 [==============================] - 1s 21ms/step - loss: 0.4696 - acc: 0.8558
Epoch 6/100
56/56 [==============================] - 1s 21ms/step - loss: 0.2113 - acc: 0.9534
Epoch 7/100
56/56 [==============================] - 1s 21ms/step - loss: 0.0945 - acc: 0.9807
Epoch 8/100
56/56 [==============================] - 1s 22ms/step - loss: 0.0834 - acc: 0.9787
Epoch 9/100
56/56 [==============================] - 1s 21ms/step - loss: 0.0605 - acc: 0.9805
Epoch 10/100
56/56 [==============================] - 1s 21ms/step - loss: 0.1375 - acc: 0.9673
Epoch 11/100
56/56 [======================

5. 인공 신경망 평가

In [17]:
# RNN 평가
score = model.evaluate(X_test, Y_test,
                       verbose = 0)

print('최종 손실값 :', score[0])
print('최종 정확도 :', score[1])

최종 손실값 : 0.27695608139038086
최종 정확도 : 0.9528089761734009
